# Introduction

- Assumptions: 
  - know qcodes basics

# The instrument server

The role of the instrument server is to manage access to instruments (both real and virtual), so they can be used from multiple processes. 
It's not strictly necessary to use it, and its designed such that using instruments through the server can be used also to talk to instruments that are instantiated directly.

Start the instrumentserver from the commandline (in an env't that has qcodes and the instrumentserver package installed) in a directory that will be the working directory for the server.

```bash
    $ instrumentserver --gui True
```

Because no instruments are instantiated at this point, we get an empty window reflecting the blank qcodes station living inside the server.
We can already create a client in this notebook that connects to the instrumentserver:

In [1]:
from instrumentserver import InstrumentClient
cli = InstrumentClient() # connect to default host (localhost) and default port (5555)

We will now see how we can make use of the fact that we can talk to the instrumentserver from multiple processes. 
We'll create a virtual instrument (something with no direct counterpart on the hardware side) that allows us to manage arbitrary experiment parameters that we can access across the setup (a fancy way to keep track of important variables). After that, we'll set up some dummy instruments and perform some fake data acquisition with them.

## Parameter manager

We'll first launch the *parameter manager*. Because it has a dedicated GUI we run it as a standalone program. 
From a command line prompt we can launch the example script that's located in the folder of this manual (again, make sure to do this within an env't that has the instrumentserver package installed):

```bash
    $ python run_parameter_manager.py
```

We'll simply get an empty window now.
The bottom of the window allows us to add arbitrary parameters and values, where dots serve as hierarchy separators (like objects and their children in python).
Let's add a few parameters, let's say `qubit.frequency`, `qubit.pipulse.len`, `qubit.pipulse.amp` and give them some arbitrary values.
We'll see something like this:

![Parameter manager window](figures/parammgr.png)

Because parameters live in the instrumentserver we can access the values here by creating a 'Proxy' instrument that forwards all requests to the 'real' instrument.

In [4]:
params = cli.get_instrument('params') # 'params' is the name the startup script gave the instrument

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()

40

**A note on synchronization of available instrument parameters between clients** 

Currently, automatic synchronization of registered parameters (and GUI-displayed values of parameters) is an outstanding feature. (This will be implemented soon.)

*It is, however, important to stress that the actual value of any parameter is stored in the server. That means that calling a parameter to get its value or to set it always results in the correct outcome.*

After we change values in the parameter manager from our interactive notebook (or a console), we need to manually refresh the GUI to make those visible:

In [5]:
# after executing, go to the GUI and press the refresh icon in the upper left to see the change.
params.qubit.pipulse.len(100)

The same goes for adding new parameters after all clients are connected.
We can add new parameters very easily from any client, but to make them visible we need to refresh the other clients:

In [6]:
# this will only be available in the GUI after pressing the refresh button.
params.add_parameter('qubit.anharmonicity', initial_value=-150.0, unit='MHz')

Conversely, if we add a parameter in the instrument GUI (let's say: `resonator.frequency`), we need to update the other clients (like the one in our notebook) before being able to access it:

In [7]:
# our proxy instrument does not yet know that the parameter exist right after creating it in the GUI.
# Only the proxy instrument of the GUI, as well as the instrumentserver, have that information at this moment.
params.resonator.frequency()

AttributeError: 'ProxyInstrumentModule' object and its delegates have no attribute 'resonator'

To sync a local client, we can call it's `update` method:

In [8]:
params.update()
params.resonator.frequency()

6.789

## Adding instruments to the server

When using 'vanilla' qcodes, we would simply create a qcodes station in our working kernel, then add instruments by instantiating the corresponding driver classes.
When using the instrumentserver to manage our instrument instances, we instead want to instantiate inside the station that the server runs.

As an example, we will add a dummy instrument that behaves like a vector network analyzer and simulates a resonator response.
Only the server will instantiate it, so instead of importing the driver, we pass the import path (of course, needs to be known in the environment that the server is running in).

In [9]:
vna = cli.create_instrument(
    'instrumentserver.testing.dummy_instruments.rf.ResonatorResponse', 
    'vna', # this is the name of the instrument in the server station
    f0=6.789e9, # the resonance frequency of the mock resonator
    df=1e6, # the linewidth
)

## Using the parameter manager in measurements

## Saving and recalling the state of the setup

## Additional perks

very easy for us to add automation features.
- logging for every instrument.
- TBD: signals to external listeners (useful for keeping track of all values without calling get)

# Plottr: data management and inspection

[Using plottr with qcodes](https://qcodes.github.io/Qcodes/examples/plotting/How-to-use-Plottr-with-QCoDeS-for-live-plotting.html)

## Saving measurement data

## Data inspection

## Live plotting

# Labcore: tools for increasing efficiency

## Parameter sweeps

### Simple example using qcodes parameters

### Measurement functions

# A practical workflow suggestion

In [ ]:
from instrumentserver import InstrumentClient

In [ ]:
ins_cli = InstrumentClient()

In [ ]:
ins_cli.list_instruments()

In [10]:
%history

from instrumentserver import InstrumentClient
cli = InstrumentClient() # connect to default host (localhost) and default port (5555)
params = cli.get_instrument('params') # 'params' is the name the startup script gave the instrument

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()
params.qubit.pipulse.len(20) # after executing, go to the GUI and press the refresh icon in the upper left to see the change.
params = cli.get_instrument('params') # 'params' is the name the startup script gave the instrument

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()
params.qubit.pipulse.len(20) # after executing, go to the GUI and press the refresh icon in the upper left to see the change.
params.add_parameter('qubit.anharmonicity', initial_value=150, unit='MHz') # this will only be available in the GUI after pressing the refresh button.
# our proxy instrument does not yet know that the parameter exist right after creating it in the GUI.
# Only

In [14]:
get_ipython().magic("history")

from instrumentserver import InstrumentClient
cli = InstrumentClient() # connect to default host (localhost) and default port (5555)
params = cli.get_instrument('params') # 'params' is the name the startup script gave the instrument

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()
params.qubit.pipulse.len(20) # after executing, go to the GUI and press the refresh icon in the upper left to see the change.
params = cli.get_instrument('params') # 'params' is the name the startup script gave the instrument

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()
params.qubit.pipulse.len(20) # after executing, go to the GUI and press the refresh icon in the upper left to see the change.
params.add_parameter('qubit.anharmonicity', initial_value=150, unit='MHz') # this will only be available in the GUI after pressing the refresh button.
# our proxy instrument does not yet know that the parameter exist right after creating it in the GUI.
# Only